In [1]:
import pororo
from pororo import Pororo

/data/ephemeral/home/shark/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
#import 모음

import nltk
import torch
from datasets import load_from_disk, DatasetDict

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
nltk.download('punkt_tab')      #'punkt'를 적을 경우 역번역 불가
mt = Pororo(task="translation", lang="multi", model='transformer.large.multi.mtpg')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /data/ephemeral/home/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [9]:
disk = "/data/ephemeral/home/level2-mrc-nlp-12/data"

train_data = load_from_disk(disk + "/train_dataset/")
train_dataset = train_data["train"]
print(len(train_data['train']))

3952


In [7]:
def back_translation(text):
    return mt(mt(text, src="ko", tgt="en"), tgt="en", src="ko")

In [ ]:
def augment_data(train_dataset, limit):
    augmented_questions = []
    for idx, example in enumerate(train_dataset):
        if idx >= limit:
            break
        original_question = example['question']
        
        #역번역
        back_translated_question = back_translation(original_question)
        
        # 새로운 예제 생성
        new_example = example.copy()
        new_example['question'] = back_translated_question
        
        augmented_questions.append(new_example)
    
    # 새로운 데이터셋 생성
    aug_train_data = DatasetDict({'train': train_dataset.add_items(augmented_questions)})
    
    return aug_train_data

In [ ]:
#제한 개수
augmentation_limit = len(train_data['train'])       #전체

aug_train_data = augment_data(train_dataset, augmentation_limit)